In [ ]:
from scipy.stats.distributions import poisson
import matplotlib.pyplot as plt
import dautil as dl
from scipy.optimize import minimize
from IPython.html.widgets.interaction import interactive
from IPython.core.display import display
from IPython.core.display import HTML

In [ ]:
lr = dl.nb.LatexRenderer(chapter=3, start=5)
lr.render(r'\!f(k; \lambda)= \Pr(X{=}k)= \frac{\lambda^k e^{-\lambda}}{k!}')
lr.render(r'\lambda=\operatorname{E}(X)=\operatorname{Var}(X)')

In [ ]:
def log_likelihood(k, mu):
    return poisson.logpmf(k, mu).sum()

In [ ]:
def count_rain_days(month):
    rain = dl.data.Weather.load()['RAIN']
    rain = (rain > 0).resample('M', how='sum')
    rain = dl.ts.groupby_month(rain)
    rain = rain.get_group(month)
    
    return rain

In [ ]:
def plot(rain, dist, params, month):
    fig, ax = plt.subplots()
    plt.title('Fitting to the Poisson distribution ({})'.format(dl.ts.short_month(month)))

    # Limiting the x-asis for a better plot
    plt.xlim([0, 15])
    plt.figtext(0.5, 0.7, 'rate {:.3f}'.format(params.x[0]), alpha=0.7,
                fontsize=14)
    plt.xlabel('# Rainy days in a month')
    plt.ylabel('Probability')
    ax.hist(dist.train, bins=dist.nbins, normed=True, label='Data')
    ax.plot(dist.x, poisson.pmf(dist.x, params.x))

In [ ]:
def fit_poisson(month):
    month_index = dl.ts.month_index(month)
    rain = count_rain_days(month_index)
    
    dist = dl.stats.Distribution(rain, poisson, range=[-0.5, 19.5])
    params = minimize(log_likelihood, x0=rain.mean(), args=(rain,))
    plot(rain, dist, params, month_index)

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
context = dl.nb.Context('fitting_poisson')
dl.nb.RcWidget(context)

In [ ]:
display(interactive(fit_poisson, month=dl.nb.create_month_widget(month='May')))
HTML(dl.report.HTMLBuilder().watermark())